In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from tqdm import tqdm

In [16]:
# 데이터 불러오기
train = pd.read_csv('../data/train.csv')

# item_id 개수 확인
train_item_id = train['item_id'].unique()

# year, month, item_id 기준으로 value 합산 (seq만 다르다면 value 합산)
monthly = (
    train
    .groupby(["item_id", "year", "month"], as_index=False)["value"]
    .sum()
)

# year, month를 하나의 키(ym)로 묶기
monthly["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
)

print("물품 개수 : ", len(train_item_id))
print("거래 횟수 : ", len(train))

물품 개수 :  100
거래 횟수 :  10836
